In [1]:
import sys; sys.path.insert(0, '../')
from PlexSim.plexsim.models import *
import networkx as nx, numpy as np
n = 64 
g = nx.grid_2d_graph(n, n, periodic = 1)

settings = dict(updateType = 'sync')
models = [
    Potts(graph = g, agentStates = [-1, 1],\
          **settings),\
    Potts(graph = g, agentStates = np.arange(0, 4).tolist(),\
          **settings),\
    RBN(graph = g, **settings),\
    SIR(graph = g, mu = .05, beta = .5, \
        **settings),\
    CCA(graph = g, agentStates = np.arange(0,4).tolist(),\
       threshold = .001, **settings),
    Percolation(graph = g, p = .5)
]

# m = models[-1]
# m.newstates.base
models[-1].states = 0
models[-1].states[models[-1].sampleNodes(1)[0,0]] = 1
T = 10
import time
start = time.time()
results = np.zeros((len(models), T, g.number_of_nodes()))
for idx, m in enumerate(models):
    results[idx] = m.simulate(T*10)[::10]
results = results.reshape(-1, T, n, n)
print(time.time() - start)

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal
%matplotlib widget
ts = IntSlider(min = 0, max = T - 1, value = 0)
fig, ax = plt.subplots(2, 3, figsize = (5,5))

fig.subplots_adjust(wspace = .12, hspace = .0001)
#setup figure
hs  = []
pad = .01
size = '5%'
titles = [m.__class__.__name__ for m in models]
for idx, (m, axi) in enumerate(zip(models, ax.flat)):
    div = mal(axi)
    cbr = div.append_axes('bottom', size = size, pad = pad)
#     div = mal(cbr)
#     cbr = div.append_axes('right', size = size, pad = pad)
    AS = models[idx].agentStates
    NS = len(AS) 
    cmap = mpl.colors.LinearSegmentedColormap.from_list(\
          None, plt.cm.viridis(np.linspace(0, 255, NS,\
                                          dtype = int)), NS)
    h = axi.imshow(results[idx, 0], cmap = cmap, vmin = AS[0],\
                  vmax = AS[-1])
    axi.set_title(titles[idx])
    axi.axis("off")
    tmp = fig.colorbar(h, cax = cbr, ticks = AS,\
                       orientation = 'horizontal')
    if idx == 3:
        tmp.set_ticklabels("Sus. Inf. Rec.".split())
    hs.append(h)
mainax = fig.add_subplot(111, frameon = 0,\
                        xticks = [], yticks = [])
mainax.set_xlabel(f'T = 0')
@interact(t = ts)
def update(t):
    for idx, h in enumerate(hs):
        h.set_data(results[idx, t])
    mainax.set_xlabel(f'T = {t}')
    fig.canvas.flush_events()
    fig.canvas.draw()
from matplotlib.animation import FuncAnimation as fan
fig.show()

<ipython-input-1-ca68e5adadb3>:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  RBN(graph = g, **settings),\


0.9895548820495605


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=9), Output()), _dom_classes=('widget-interact',)…

In [9]:
m.__class__.__name__
#an = fan(fig, update, frames = np.arange(0, int(T*1), 10), interval = 100,\
#         blit = True) 
#an.save('PlexSim_banner.mp4', writer = 'ffmpeg')

'CCA'